# IMPORTS

In [ ]:
import logging
logging.basicConfig(level=logging.WARN, format='%(asctime)s - %(levelname)s - %(message)s') # NOTSET, DEBUG, INFO, WARN, ERROR, CRITICAL

import os, sys
import torch
import numpy as np

import CL_inference as cl_inference
N_threads = cl_inference.train_tools.set_N_threads_(N_threads=1)
torch.set_num_threads(N_threads)
torch.set_num_interop_threads(N_threads)
device = cl_inference.train_tools.set_torch_device_()

%load_ext autoreload

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib notebook
plt.style.use('default')
plt.close('all')
font, rcnew = cl_inference.plot_utils.matplotlib_default_config()
mpl.rc('font', **font)
plt.rcParams.update(rcnew)
plt.style.use('tableau-colorblind10')
%config InlineBackend.figure_format = 'retina'

# Set load path and define thresholds_bias employed

In [ ]:
models_path = '/cosmos_storage/home/dlopez/Projects/CL_inference/models'
thresholds_bias = np.linspace(0.5, 6, 20)

In [ ]:
ii_stored_model = 0

# Scale cut tests

### Train inference cosmology + baryon parameters

In [ ]:
custom_titles, limits_plots_inference, list_range_priors = cl_inference.plot_utils.get_titles_limits_and_priors(True)

In [ ]:
fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors = cl_inference.evaluation_tools.wrapper_load_numpy_stored_summaries_kcuts(models_path)

In [ ]:
selected_sigma = 3.
idx = (np.abs(thresholds_bias - selected_sigma)).argmin()

fontsize=26
fontsize1=18
fig, axs = plt.subplots(1, len(custom_titles), figsize=(4.2*len(custom_titles), 5.2))
axs[0].set_ylabel(r'Fraction bias >' + str(selected_sigma) + '$\sigma$', size=fontsize)
for ii_cosmo_param in range(len(custom_titles)):
    ax = axs[ii_cosmo_param]
    ax.set_xscale('log')
    ax.set_ylim([-0.001, .015])
    ax.set_title(custom_titles[ii_cosmo_param], size=fontsize+8, pad=16)
    ax.set_xlabel(r'$k_\mathrm{cut} \left[ h\, \mathrm{Mpc}^{-1} \right]$', size=fontsize)
    if ii_cosmo_param != 0:
        ax.set_yticks([])
    ax.axhline(0, c='k', ls=':', lw=0.9)
        
    mode_name = "all also baryons"
    color = 'k'
    tmp_median_mean = np.zeros(len(fraction_biased[mode_name].keys()))
    tmp_median_std = np.zeros(len(fraction_biased[mode_name].keys()))
    scale_cuts = np.zeros(len(fraction_biased[mode_name].keys()))
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        scale_cuts[kk] = 10**float(name_load)
        tmp_median_mean[kk] = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)[idx]
        tmp_median_std[kk] = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)[idx]
    ax.scatter(scale_cuts, tmp_median_mean, color=color, s=20)
    ax.errorbar(scale_cuts, tmp_median_mean, yerr=tmp_median_std, color=color, ls='', capsize=2, alpha=.9, elinewidth=1.)
    ax.fill_between(scale_cuts, tmp_median_mean-tmp_median_std, tmp_median_mean+tmp_median_std, color=color, alpha=0.3)
    
fig.set_tight_layout(True)
mpl.pyplot.show()

In [ ]:
fontsize=26
fontsize1=18
fig, axs = plt.subplots(1, len(custom_titles), figsize=(4.2*len(custom_titles), 5.2))
axs[0].set_ylabel(r'$\frac{2\sigma}{\Delta \mathrm{Prior}}$', size=fontsize)
for ii_cosmo_param in range(len(custom_titles)):
    ax = axs[ii_cosmo_param]
    ax.set_xscale('log')
    ax.set_title(custom_titles[ii_cosmo_param], size=fontsize+8, pad=16)
    ax.set_xlabel(r'$k_\mathrm{cut} \left[ h\, \mathrm{Mpc}^{-1} \right]$', size=fontsize)
    ax.axhline(0, c='k', ls=':', lw=0.9)
    if ii_cosmo_param != 0:
        ax.set_yticks([])
    ax.set_ylim([-0.01, 0.61])
    
    tmp_prior_range = (list_range_priors[ii_cosmo_param][1]-list_range_priors[ii_cosmo_param][0])
    
    mode_name = "all also baryons"
    color = 'k'
    tmp_median_err_mean = np.zeros(len(fraction_biased[mode_name].keys()))
    tmp_median_err_std = np.zeros(len(fraction_biased[mode_name].keys()))
    scale_cuts = np.zeros(len(fraction_biased[mode_name].keys()))
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        scale_cuts[kk] = 10**float(name_load)
        tmp_median_err_mean[kk] = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
        tmp_median_err_std[kk] = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.scatter(scale_cuts, tmp_median_err_mean, color=color, s=20)
    ax.errorbar(scale_cuts, tmp_median_err_mean, yerr=tmp_median_err_std, color=color, ls='',capsize=2, alpha=.9, elinewidth=1.)
    ax.fill_between(scale_cuts, tmp_median_err_mean-tmp_median_err_std, tmp_median_err_mean+tmp_median_err_std, color=color, alpha=0.3)

fig.set_tight_layout(True)
mpl.pyplot.show()

### Scale cuts comparison different training sets

In [ ]:
custom_titles, limits_plots_inference, list_range_priors = cl_inference.plot_utils.get_titles_limits_and_priors(False)

In [ ]:
selected_sigma = 3.
idx = (np.abs(thresholds_bias - selected_sigma)).argmin()

fontsize=26
fontsize1=18
fig, axs = plt.subplots(1, len(custom_titles), figsize=(4.2*len(custom_titles), 5.2))
axs[0].set_ylabel(r'Fraction bias >' + str(selected_sigma) + '$\sigma$', size=fontsize)
for ii_cosmo_param in range(len(custom_titles)):
    ax = axs[ii_cosmo_param]
    ax.set_xscale('log')
    ax.set_ylim([-0.05, 1.])
    ax.set_title(custom_titles[ii_cosmo_param], size=fontsize+8, pad=16)
    ax.set_xlabel(r'$k_\mathrm{cut} \left[ h\, \mathrm{Mpc}^{-1} \right]$', size=fontsize)
    if ii_cosmo_param != 0:
        ax.set_yticks([])
    ax.axhline(0, c='k', ls=':', lw=0.9)
        
    mode_name = "all also baryons"
    color = 'k'
    tmp_median_mean = np.zeros(len(fraction_biased[mode_name].keys()))
    tmp_median_std = np.zeros(len(fraction_biased[mode_name].keys()))
    scale_cuts = np.zeros(len(fraction_biased[mode_name].keys()))
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        scale_cuts[kk] = 10**float(name_load)
        tmp_median_mean[kk] = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)[idx]
        tmp_median_std[kk] = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)[idx]
    ax.scatter(scale_cuts, tmp_median_mean, color=color, s=20)
    ax.errorbar(scale_cuts, tmp_median_mean, yerr=tmp_median_std, color=color, ls='', capsize=2, alpha=.9, elinewidth=1.)
    ax.fill_between(scale_cuts, tmp_median_mean-tmp_median_std, tmp_median_mean+tmp_median_std, color=color, alpha=0.3)
    
    mode_name = "all not baryons"
    color = 'grey'
    tmp_median_mean = np.zeros(len(fraction_biased[mode_name].keys()))
    tmp_median_std = np.zeros(len(fraction_biased[mode_name].keys()))
    scale_cuts = np.zeros(len(fraction_biased[mode_name].keys()))
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        scale_cuts[kk] = 10**float(name_load)
        tmp_median_mean[kk] = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)[idx]
        tmp_median_std[kk] = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)[idx]
    ax.scatter(scale_cuts, tmp_median_mean, color=color, s=20)
    ax.errorbar(scale_cuts, tmp_median_mean, yerr=tmp_median_std, color=color, ls='', capsize=2, alpha=.9, elinewidth=1.)
    ax.fill_between(scale_cuts, tmp_median_mean-tmp_median_std, tmp_median_mean+tmp_median_std, color=color, alpha=0.3)
    
    mode_name = "no CL"
    color = 'red'
    tmp_median_mean = np.zeros(len(fraction_biased[mode_name].keys()))
    tmp_median_std = np.zeros(len(fraction_biased[mode_name].keys()))
    scale_cuts = np.zeros(len(fraction_biased[mode_name].keys()))
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        scale_cuts[kk] = 10**float(name_load)
        tmp_median_mean[kk] = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)[idx]
        tmp_median_std[kk] = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)[idx]
    ax.scatter(scale_cuts, tmp_median_mean, color=color, s=20)
    ax.errorbar(scale_cuts, tmp_median_mean, yerr=tmp_median_std, color=color, ls='', capsize=2, alpha=.9, elinewidth=1.)
    ax.fill_between(scale_cuts, tmp_median_mean-tmp_median_std, tmp_median_mean+tmp_median_std, color=color, alpha=0.3)
    
    mode_name = "CL Wein"
    color = 'royalblue'
    tmp_median_mean = np.zeros(len(fraction_biased[mode_name].keys()))
    tmp_median_std = np.zeros(len(fraction_biased[mode_name].keys()))
    scale_cuts = np.zeros(len(fraction_biased[mode_name].keys()))
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        scale_cuts[kk] = 10**float(name_load)
        tmp_median_mean[kk] = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)[idx]
        tmp_median_std[kk] = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)[idx]
    ax.scatter(scale_cuts, tmp_median_mean, color=color, s=20)
    ax.errorbar(scale_cuts, tmp_median_mean, yerr=tmp_median_std, color=color, ls='', capsize=2, alpha=.9, elinewidth=1.)
    ax.fill_between(scale_cuts, tmp_median_mean-tmp_median_std, tmp_median_mean+tmp_median_std, color=color, alpha=0.3)
    
    mode_name = "CL VICReg"
    color = 'limegreen'
    tmp_median_mean = np.zeros(len(fraction_biased[mode_name].keys()))
    tmp_median_std = np.zeros(len(fraction_biased[mode_name].keys()))
    scale_cuts = np.zeros(len(fraction_biased[mode_name].keys()))
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        scale_cuts[kk] = 10**float(name_load)
        tmp_median_mean[kk] = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)[idx]
        tmp_median_std[kk] = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)[idx]
    ax.scatter(scale_cuts, tmp_median_mean, color=color, s=20)
    ax.errorbar(scale_cuts, tmp_median_mean, yerr=tmp_median_std, color=color, ls='', capsize=2, alpha=.9, elinewidth=1.)
    ax.fill_between(scale_cuts, tmp_median_mean-tmp_median_std, tmp_median_mean+tmp_median_std, color=color, alpha=0.3)
    
custom_lines = [
    mpl.lines.Line2D([0],[0],color='k',ls='-',lw=3,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='grey',ls='-',lw=3,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='red',ls='-',lw=3,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='royalblue',ls='-',lw=3,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='limegreen',ls='-',lw=3,marker=None,markersize=8)
]
custom_labels = [
    'No CL Dset.-all inference cosmo. & baryons',
    'No CL Dset.-all inference only cosmo.',
    'No CL Dset.-illsutris-eagle',
    'CL Wein. Dset.-illsutris-eagle',
    'CL VICReg Dset.-illsutris-eagle'
]
legend = axs[0].legend(custom_lines, custom_labels, loc='upper left', fancybox=True, shadow=True, ncol=1,fontsize=10)
axs[0].add_artist(legend)

fig.set_tight_layout(True)
mpl.pyplot.show()

In [ ]:
fontsize=26
fontsize1=18
fig, axs = plt.subplots(1, len(custom_titles), figsize=(4.2*len(custom_titles), 5.2))
axs[0].set_ylabel(r'$\frac{2\sigma}{\Delta \mathrm{Prior}}$', size=fontsize)
for ii_cosmo_param in range(len(custom_titles)):
    ax = axs[ii_cosmo_param]
    ax.set_xscale('log')
    ax.set_title(custom_titles[ii_cosmo_param], size=fontsize+8, pad=16)
    ax.set_xlabel(r'$k_\mathrm{cut} \left[ h\, \mathrm{Mpc}^{-1} \right]$', size=fontsize)
    ax.axhline(0, c='k', ls=':', lw=0.9)
    if ii_cosmo_param != 0:
        ax.set_yticks([])
    ax.set_ylim([-0.01, 0.61])
        
    tmp_prior_range = (list_range_priors[ii_cosmo_param][1]-list_range_priors[ii_cosmo_param][0])
    
    mode_name = "all also baryons"
    color = 'k'
    tmp_median_err_mean = np.zeros(len(fraction_biased[mode_name].keys()))
    tmp_median_err_std = np.zeros(len(fraction_biased[mode_name].keys()))
    scale_cuts = np.zeros(len(fraction_biased[mode_name].keys()))
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        scale_cuts[kk] = 10**float(name_load)
        tmp_median_err_mean[kk] = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
        tmp_median_err_std[kk] = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.scatter(scale_cuts, tmp_median_err_mean, color=color, s=20)
    ax.errorbar(scale_cuts, tmp_median_err_mean, yerr=tmp_median_err_std, color=color, ls='',capsize=2, alpha=.9, elinewidth=1.)
    ax.fill_between(scale_cuts, tmp_median_err_mean-tmp_median_err_std, tmp_median_err_mean+tmp_median_err_std, color=color, alpha=0.3)
    
    mode_name = "all not baryons"
    color = 'grey'
    tmp_median_err_mean = np.zeros(len(fraction_biased[mode_name].keys()))
    tmp_median_err_std = np.zeros(len(fraction_biased[mode_name].keys()))
    scale_cuts = np.zeros(len(fraction_biased[mode_name].keys()))
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        scale_cuts[kk] = 10**float(name_load)
        tmp_median_err_mean[kk] = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
        tmp_median_err_std[kk] = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.scatter(scale_cuts, tmp_median_err_mean, color=color, s=20)
    ax.errorbar(scale_cuts, tmp_median_err_mean, yerr=tmp_median_err_std, color=color, ls='',capsize=2, alpha=.9, elinewidth=1.)
    ax.fill_between(scale_cuts, tmp_median_err_mean-tmp_median_err_std, tmp_median_err_mean+tmp_median_err_std, color=color, alpha=0.3)
    
    mode_name = "no CL"
    color = 'red'
    tmp_median_err_mean = np.zeros(len(fraction_biased[mode_name].keys()))
    tmp_median_err_std = np.zeros(len(fraction_biased[mode_name].keys()))
    scale_cuts = np.zeros(len(fraction_biased[mode_name].keys()))
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        scale_cuts[kk] = 10**float(name_load)
        tmp_median_err_mean[kk] = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
        tmp_median_err_std[kk] = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.scatter(scale_cuts, tmp_median_err_mean, color=color, s=20)
    ax.errorbar(scale_cuts, tmp_median_err_mean, yerr=tmp_median_err_std, color=color, ls='',capsize=2, alpha=.9, elinewidth=1.)
    ax.fill_between(scale_cuts, tmp_median_err_mean-tmp_median_err_std, tmp_median_err_mean+tmp_median_err_std, color=color, alpha=0.3)
    
    mode_name = "CL Wein"
    color = 'royalblue'
    tmp_median_err_mean = np.zeros(len(fraction_biased[mode_name].keys()))
    tmp_median_err_std = np.zeros(len(fraction_biased[mode_name].keys()))
    scale_cuts = np.zeros(len(fraction_biased[mode_name].keys()))
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        scale_cuts[kk] = 10**float(name_load)
        tmp_median_err_mean[kk] = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
        tmp_median_err_std[kk] = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.scatter(scale_cuts, tmp_median_err_mean, color=color, s=20)
    ax.errorbar(scale_cuts, tmp_median_err_mean, yerr=tmp_median_err_std, color=color, ls='',capsize=2, alpha=.9, elinewidth=1.)
    ax.fill_between(scale_cuts, tmp_median_err_mean-tmp_median_err_std, tmp_median_err_mean+tmp_median_err_std, color=color, alpha=0.3)
    
    mode_name = "CL VICReg"
    color = 'limegreen'
    tmp_median_err_mean = np.zeros(len(fraction_biased[mode_name].keys()))
    tmp_median_err_std = np.zeros(len(fraction_biased[mode_name].keys()))
    scale_cuts = np.zeros(len(fraction_biased[mode_name].keys()))
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        scale_cuts[kk] = 10**float(name_load)
        tmp_median_err_mean[kk] = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
        tmp_median_err_std[kk] = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.scatter(scale_cuts, tmp_median_err_mean, color=color, s=20)
    ax.errorbar(scale_cuts, tmp_median_err_mean, yerr=tmp_median_err_std, color=color, ls='',capsize=2, alpha=.9, elinewidth=1.)
    ax.fill_between(scale_cuts, tmp_median_err_mean-tmp_median_err_std, tmp_median_err_mean+tmp_median_err_std, color=color, alpha=0.3)
        
custom_lines = [
    mpl.lines.Line2D([0],[0],color='k',ls='-',lw=3,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='grey',ls='-',lw=3,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='red',ls='-',lw=3,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='royalblue',ls='-',lw=3,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='limegreen',ls='-',lw=3,marker=None,markersize=8)
]
custom_labels = [
    'No CL Dset.-all inference cosmo. & baryons',
    'No CL Dset.-all inference only cosmo.',
    'No CL Dset.-illsutris-eagle',
    'CL Wein. Dset.-illsutris-eagle',
    'CL VICReg Dset.-illsutris-eagle'
]
legend = axs[0].legend(custom_lines, custom_labels, loc='upper left', fancybox=True, shadow=True, ncol=1,fontsize=10)
axs[0].add_artist(legend)

fig.set_tight_layout(True)
mpl.pyplot.show()

# Compare different training modes

In [ ]:
fraction_biased = {}
NN_points = {}
bin_centers = {}
y_hists = {}
bin_centers_err = {}
y_hists_err = {}
median_err = {}
std_err = {}
colors = {}

default_colors = None

fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors = cl_inference.evaluation_tools.load_numpy_stored_summaries(
    models_path, "all", {"all" : "only_inference_models_all_kmax_0.6"}, fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors, default_colors
)

fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors = cl_inference.evaluation_tools.load_numpy_stored_summaries(
    models_path, "no CL", {"illustris-eagle" : "only_inference_models_illustris_eagle_kmax_0.6"}, fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors, default_colors
)

fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors = cl_inference.evaluation_tools.load_numpy_stored_summaries(
    models_path, "no CL", {"illustris-eagle" : "only_inference_models_illustris_eagle_kmax_0.6"}, fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors, default_colors
)

fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors = cl_inference.evaluation_tools.load_numpy_stored_summaries(
    models_path, "CL Wein", {
        "Disjoint training"         : "only_inference_CL_Wein_models_illustris_eagle_kmax_0.6",
        "Joint train. from scratch" : "join_inference_CL_Wein_models_illustris_eagle_kmax_0.6",
        "Joint train. from reload"  : "join_reload_inference_CL_Wein_models_illustris_eagle_kmax_0.6"
    }, fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors, default_colors
)

fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors = cl_inference.evaluation_tools.load_numpy_stored_summaries(
    models_path, "CL VICReg", {
        "Disjoint training"         : "only_inference_CL_VICReg_models_illustris_eagle_kmax_0.6",
        "Joint train. from scratch" : "join_inference_CL_VICReg_models_illustris_eagle_kmax_0.6",
        "Joint train. from reload"  : "join_reload_inference_CL_VICReg_models_illustris_eagle_kmax_0.6"
    }, fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors, default_colors
)

In [ ]:
list_linestyles = ['-', ':', '-.']

In [ ]:
fontsize=26
fontsize1=18
fig, axs = plt.subplots(1, len(custom_titles), figsize=(4.2*len(custom_titles), 5.2))
axs[0].set_ylabel(r'Fraction biased points ', size=fontsize)
for ii_cosmo_param in range(len(custom_titles)):
    ax = axs[ii_cosmo_param]
    ax.set_xlim([1.5, 5.])
    ax.set_ylim([-0.1, 1])
    ax.set_title(custom_titles[ii_cosmo_param], size=fontsize+8, pad=16)
    ax.set_xlabel(r'$\sigma_\mathrm{thr.}$', size=fontsize)
    ax.axhline(0, c='k', ls=':', lw=0.9)
    if ii_cosmo_param != 0:
        ax.set_yticks([])
    
    mode_name = "all"
    color = 'grey'
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        ax.plot(thresholds_bias, tmp_mean, color=color, ls='-', lw=2)
        ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color, alpha=0.3)
        
    mode_name = "no CL"
    color = 'red'
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        ax.plot(thresholds_bias, tmp_mean, color=color, ls='--', lw=2)
        ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color, alpha=0.3)
        
    mode_name = "CL Wein"
    color = 'royalblue'
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        ax.plot(thresholds_bias, tmp_mean, color=color, ls=list_linestyles[kk], lw=2)
        ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color, alpha=0.3)
        
    mode_name = "CL VICReg"
    color = 'limegreen'
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        ax.plot(thresholds_bias, tmp_mean, color=color, ls=list_linestyles[kk], lw=2)
        ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color, alpha=0.3)

custom_lines = [
    mpl.lines.Line2D([0],[0],color='grey',ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='red',ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='royalblue',ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='limegreen',ls='-',lw=8,marker=None,markersize=8)
]
custom_labels = [
    'No CL Dset.-all inference only cosmo.',
    'No CL Dset.-illsutris-eagle',
    'CL Wein. Dset.-illsutris-eagle',
    'CL VICReg Dset.-illsutris-eagle'
]
legend = axs[0].legend(custom_lines, custom_labels, loc='upper right', fancybox=True, shadow=True, ncol=1,fontsize=10)
axs[0].add_artist(legend)

custom_lines = [
    mpl.lines.Line2D([0],[0],color='grey',ls='--',lw=2,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='grey',ls='-',lw=2,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='grey',ls=':',lw=2,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='grey',ls='-.',lw=2,marker=None,markersize=8)
]
custom_labels = [
    'No CL',
    'Disjoint training',
    'Joint train. from scratch',
    'Joint train. from reload'
]
legend = axs[-1].legend(custom_lines, custom_labels, loc='upper right', fancybox=True, shadow=True, ncol=1,fontsize=10)
axs[-1].add_artist(legend)
    
fig.set_tight_layout(True)
mpl.pyplot.show()

In [ ]:
max_err_hist = [0.05, 0.012, 0.12, 0.042, 0.06, 3.2, 1.5, 1.5, 3., .4, 1., 3.]

In [ ]:
ii_batch_sample = 0

fontsize=26
fontsize1=18
fig, axs = plt.subplots(1, len(custom_titles), figsize=(5.2*len(custom_titles), 5.2))
axs[0].set_ylabel(r'Normalized Counts ', size=fontsize)
for ii_cosmo_param in range(len(custom_titles)):
    
    tmp_prior_range = (list_range_priors[ii_cosmo_param][1]-list_range_priors[ii_cosmo_param][0])
    
    ax = axs[ii_cosmo_param]
    ax.set_title(custom_titles[ii_cosmo_param], size=fontsize+8, pad=16)
    ax.set_xlabel(r'$\mathrm{median}\left(\frac{2\sigma}{\Delta \mathrm{Prior}}\right)$', size=fontsize)
#     ax.set_xlim([0.0, max_err_hist[ii_cosmo_param]/tmp_prior_range])
    
    mode_name = "all"
    color = 'grey'
    for kk, name_load in enumerate(median_err[mode_name].keys()):
#         ax.plot(
#             bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param]/tmp_prior_range,
#             y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param][ii_batch_sample]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param],
#             color=color, lw=0.2, alpha=0.9
#         )
        tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
        tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
        ax.axvline(tmp_mean, color=color, ls='-', lw=2)
        ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color)
        
    mode_name = "no CL"
    color = 'red'
    for kk, name_load in enumerate(median_err[mode_name].keys()):
#         ax.plot(
#             bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param]/tmp_prior_range,
#             y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param][ii_batch_sample]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param],
#             color=color, lw=0.2, alpha=0.9
#         )
        tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
        tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
        ax.axvline(tmp_mean, color=color, ls='-', lw=2)
        ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color)
        
    mode_name = "CL Wein"
    color = 'royalblue'
    for kk, name_load in enumerate(median_err[mode_name].keys()):
#         ax.plot(
#             bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param]/tmp_prior_range,
#             y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param][ii_batch_sample]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param],
#             color=color, lw=0.2, alpha=0.9
#         )
        tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
        tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
        ax.axvline(tmp_mean, color=color, ls=list_linestyles[kk], lw=2)
        ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color)
        
    mode_name = "CL VICReg"
    color = 'limegreen'
    for kk, name_load in enumerate(median_err[mode_name].keys()):
#         ax.plot(
#             bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param]/tmp_prior_range,
#             y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param][ii_batch_sample]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param],
#             color=color, lw=0.2, alpha=0.9
#         )
        tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
        tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
        ax.axvline(tmp_mean, color=color, ls=list_linestyles[kk], lw=2)
        ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color)
        
custom_lines = [
    mpl.lines.Line2D([0],[0],color='grey',ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='red',ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='royalblue',ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='limegreen',ls='-',lw=8,marker=None,markersize=8)
]
custom_labels = [
    'No CL Dset.-all inference only cosmo.',
    'No CL Dset.-illsutris-eagle',
    'CL Wein. Dset.-illsutris-eagle',
    'CL VICReg Dset.-illsutris-eagle'
]
legend = axs[0].legend(custom_lines, custom_labels, loc='upper right', fancybox=True, shadow=True, ncol=1,fontsize=10)
axs[0].add_artist(legend)

custom_lines = [
    mpl.lines.Line2D([0],[0],color='grey',ls='--',lw=2,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='grey',ls='-',lw=2,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='grey',ls=':',lw=2,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='grey',ls='-.',lw=2,marker=None,markersize=8)
]
custom_labels = [
    'No CL',
    'Disjoint training',
    'Joint train. from scratch',
    'Joint train. from reload'
]
legend = axs[-1].legend(custom_lines, custom_labels, loc='upper right', fancybox=True, shadow=True, ncol=1,fontsize=10)
axs[-1].add_artist(legend)
        
fig.set_tight_layout(True)
mpl.pyplot.show()

# Check bias and errors comparing with the training dataset

In [ ]:
fontsize=26
fontsize1=18
fig, axs = plt.subplots(1, len(custom_titles), figsize=(4.2*len(custom_titles), 5.2))
axs[0].set_ylabel(r'Fraction biased points ', size=fontsize)
for ii_cosmo_param in range(len(custom_titles)):
    ax = axs[ii_cosmo_param]
    ax.set_xlim([1.5, 5.])
    ax.set_ylim([-0.1, 1])
    ax.set_title(custom_titles[ii_cosmo_param], size=fontsize+8, pad=16)
    ax.set_xlabel(r'$\sigma_\mathrm{thr.}$', size=fontsize)
    ax.axhline(0, c='k', ls=':', lw=0.9)
    if ii_cosmo_param != 0:
        ax.set_yticks([])
    
    mode_name = "no CL"
    name_load = "illustris-eagle"
    linestyle = '--'
    color0 = 'grey'
    color1 = '#9467BD'
    color2 = '#D62728'
    
    tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
    tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
    ax.plot(thresholds_bias, tmp_mean, color=color0, ls=linestyle, lw=2)
    ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color0, alpha=0.3)
    
    ii_stored_model_train = 1
    tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    ax.plot(thresholds_bias, tmp_mean, color=color1, ls='-', lw=2)
    ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color1, alpha=0.3)
    
    ii_stored_model_train = 2
    tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    ax.plot(thresholds_bias, tmp_mean, color=color2, ls='-', lw=2)
    ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color2, alpha=0.3)
    
        
    mode_name = "CL Wein"
    name_load = "Disjoint training"
    linestyle = '-'
    
    tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
    tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
    ax.plot(thresholds_bias, tmp_mean, color=color0, ls='-', lw=2)
    ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color0, alpha=0.3)

    ii_stored_model_train = 1
    tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    ax.plot(thresholds_bias, tmp_mean, color=color1, ls='-', lw=2)
    ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color1, alpha=0.3)
    
    ii_stored_model_train = 2
    tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    ax.plot(thresholds_bias, tmp_mean, color=color2, ls='-', lw=2)
    ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color2, alpha=0.3)
        
custom_lines = [
    mpl.lines.Line2D([0],[0],color=color0,ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color=color1,ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color=color2, ls='-',lw=8,marker=None,markersize=8)
]
custom_labels = [
    'Dset.-all',
    'Dset.-illsutris',
    'Dset.-eagle'
]
legend = axs[0].legend(custom_lines, custom_labels, loc='upper right', fancybox=True, shadow=True, ncol=1,fontsize=12)
axs[0].add_artist(legend)

custom_lines = [
    mpl.lines.Line2D([0],[0],color='grey',ls='--',lw=2,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='grey',ls='-',lw=2,marker=None,markersize=8)
]
custom_labels = [
    'No CL',
    'CL ' + name_load
]
legend = axs[-1].legend(custom_lines, custom_labels, loc='upper right', fancybox=True, shadow=True, ncol=1,fontsize=12)
axs[-1].add_artist(legend)
        
fig.set_tight_layout(True)
mpl.pyplot.show()

In [ ]:
ii_batch_sample = 0

fontsize=26
fontsize1=18
fig, axs = plt.subplots(1, len(custom_titles), figsize=(5.2*len(custom_titles), 5.2))
axs[0].set_ylabel(r'Normalized Counts ', size=fontsize)
for ii_cosmo_param in range(len(custom_titles)):
    
    tmp_prior_range = (list_range_priors[ii_cosmo_param][1]-list_range_priors[ii_cosmo_param][0])
    
    ax = axs[ii_cosmo_param]
    ax.set_title(custom_titles[ii_cosmo_param], size=fontsize+8, pad=16)
    ax.set_xlabel(r'$\mathrm{median}\left(\frac{2\sigma}{\Delta \mathrm{Prior}}\right)$', size=fontsize)
#     ax.set_xlim([0.0, max_err_hist[ii_cosmo_param]/tmp_prior_range])
    
    mode_name = "no CL"
    name_load = "illustris-eagle"
    linestyle = '--'
    color0 = 'grey'
    color1 = '#9467BD'
    color2 = '#D62728'
    
#     ax.plot(
#         bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param]/tmp_prior_range,
#         y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param],
#         color=color0, lw=0.2, alpha=0.9, ls=linestyle
#     )
    tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
    tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.axvline(tmp_mean, color=color0, ls=linestyle, lw=2)
    ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color0)
    
    ii_stored_model_train = 1
#     ax.plot(
#         bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range,
#         y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param],
#         color=color1, lw=0.2, alpha=0.9, ls=linestyle
#     )
    tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.axvline(tmp_mean, color=color1, ls=linestyle, lw=2)
    ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color1)
    
    ii_stored_model_train = 2
#     ax.plot(
#         bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range,
#         y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param],
#         color=color2, lw=0.2, alpha=0.9, ls=linestyle
#     )
    tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.axvline(tmp_mean, color=color2, ls=linestyle, lw=2)
    ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color2)
        
        
    mode_name = "CL Wein"
    name_load = "Disjoint training"
    linestyle = '-'
    
#     ax.plot(
#         bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param]/tmp_prior_range,
#         y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param],
#         color=color0, lw=0.2, alpha=0.9, ls=linestyle
#     )
    tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
    tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.axvline(tmp_mean, color=color0, ls=linestyle, lw=2)
    ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color0)
        
    ii_stored_model_train = 1
#     ax.plot(
#         bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range,
#         y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param],
#         color=color1, lw=0.2, alpha=0.9, ls=linestyle
#     )
    tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.axvline(tmp_mean, color=color1, ls=linestyle, lw=2)
    ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color1)
    
    ii_stored_model_train = 2
#     ax.plot(
#         bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range,
#         y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param],
#         color=color2, lw=0.2, alpha=0.9, ls=linestyle
#     )
    tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.axvline(tmp_mean, color=color2, ls=linestyle, lw=2)
    ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color2)

    
custom_lines = [
    mpl.lines.Line2D([0],[0],color=color0,ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color=color1,ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color=color2, ls='-',lw=8,marker=None,markersize=8)
]
custom_labels = [
    'Dset.-all',
    'Dset.-illsutris',
    'Dset.-eagle'
]
legend = axs[0].legend(custom_lines, custom_labels, loc='upper right', fancybox=True, shadow=True, ncol=1,fontsize=12)
axs[0].add_artist(legend)

custom_lines = [
    mpl.lines.Line2D([0],[0],color='grey',ls='--',lw=2,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='grey',ls='-',lw=2,marker=None,markersize=8)
]
custom_labels = [
    'No CL',
    'CL ' + name_load
]
legend = axs[-1].legend(custom_lines, custom_labels, loc='upper right', fancybox=True, shadow=True, ncol=1,fontsize=12)
axs[-1].add_artist(legend)


fig.set_tight_layout(True)
mpl.pyplot.show()

# Comparison train different models $k_{cut} = 0.6$

In [ ]:
custom_titles, limits_plots_inference, list_range_priors = cl_inference.plot_utils.get_titles_limits_and_priors(False)

In [ ]:
fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors = cl_inference.evaluation_tools.wrapper_load_numpy_stored_summaries(models_path)

In [ ]:
selected_plot_keys = ['eagle-bahamas', 'v1-v2']
# selected_plot_keys = fraction_biased[mode_name].keys()

fontsize=26
fontsize1=18
fig, axs = plt.subplots(1, len(custom_titles), figsize=(4.2*len(custom_titles), 5.2))
axs[0].set_ylabel(r'Fraction biased points ', size=fontsize)
custom_lines = []
custom_labels = []
custom_lines1 = []
custom_labels1 = []
for ii_cosmo_param in range(len(custom_titles)):
    ax = axs[ii_cosmo_param]
    ax.set_xlim([1.5, 5.])
    ax.set_ylim([-0.1, 1])
    ax.set_title(custom_titles[ii_cosmo_param], size=fontsize+8, pad=16)
    ax.set_xlabel(r'$\sigma_\mathrm{thr.}$', size=fontsize)
    ax.axhline(0, c='k', ls=':', lw=0.9)
    if ii_cosmo_param != 0:
        ax.set_yticks([])
    
    mode_name = "all"
    for kk, name_load in enumerate(fraction_biased[mode_name].keys()):
        color = colors[mode_name][name_load]
        for jj in range(fraction_biased[mode_name][name_load].shape[0]):
            ax.plot(thresholds_bias, fraction_biased[mode_name][name_load][jj, ii_stored_model, ii_cosmo_param], color=color, lw=0.2, alpha=0.9)
        tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        ax.scatter(thresholds_bias, tmp_mean, color=color, s=20)
        ax.errorbar(thresholds_bias, tmp_mean, yerr=tmp_std, color=color, ls='-',capsize=2, alpha=.9, elinewidth=1.)
        ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color, alpha=0.3)
        if ii_cosmo_param == 0:
            custom_lines.append(mpl.lines.Line2D([0], [0], color=color, ls='-', lw=9, marker=None, markersize=9))
            custom_labels.append(name_load)
            if kk == 0:
                custom_lines1.append(mpl.lines.Line2D([0], [0], color='grey', ls='-', lw=1., marker=None, markersize=9))
                custom_labels1.append(mode_name)
            
    mode_name = "no CL"
    for kk, name_load in enumerate(selected_plot_keys):
        color = colors[mode_name][name_load]
        for jj in range(fraction_biased[mode_name][name_load].shape[0]):
            ax.plot(thresholds_bias, fraction_biased[mode_name][name_load][jj, ii_stored_model, ii_cosmo_param], color=color, lw=0.2, alpha=0.9)
        tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        ax.scatter(thresholds_bias, tmp_mean, color=color, s=20)
        ax.errorbar(thresholds_bias, tmp_mean, yerr=tmp_std, color=color, ls='--', capsize=2, alpha=.9, elinewidth=1.)
        ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color, alpha=0.3)
        if ii_cosmo_param == 0:
            custom_lines.append(mpl.lines.Line2D([0], [0], color=color, ls='--', lw=9, marker=None, markersize=9))
            custom_labels.append(name_load)
            if kk == 0:
                custom_lines1.append(mpl.lines.Line2D([0], [0], color='grey', ls='-', lw=1., marker=None, markersize=9))
                custom_labels1.append(mode_name)
            
    mode_name = "CL Wein"
    for kk, name_load in enumerate(selected_plot_keys):
        color = colors[mode_name][name_load]
        for jj in range(fraction_biased[mode_name][name_load].shape[0]):
            ax.plot(thresholds_bias, fraction_biased[mode_name][name_load][jj, ii_stored_model, ii_cosmo_param], color=color, lw=0.2, alpha=0.9)
        tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        ax.scatter(thresholds_bias, tmp_mean, color=color, s=20)
        ax.errorbar(thresholds_bias, tmp_mean, yerr=tmp_std, color=color, ls='-', capsize=2, alpha=.9, elinewidth=1.)
        ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color, alpha=0.3)
        if ii_cosmo_param == 0 and kk == 0:
            custom_lines1.append(mpl.lines.Line2D([0], [0], color='grey', ls='-', lw=1., marker=None, markersize=9))
            custom_labels1.append(mode_name)
        
    mode_name = "CL VICReg"
    for kk, name_load in enumerate(selected_plot_keys):
        color = colors[mode_name][name_load]
        for jj in range(fraction_biased[mode_name][name_load].shape[0]):
            ax.plot(thresholds_bias, fraction_biased[mode_name][name_load][jj, ii_stored_model, ii_cosmo_param], color=color, lw=0.2, alpha=0.9)
        tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        ax.scatter(thresholds_bias, tmp_mean, color=color, s=20)
        ax.errorbar(thresholds_bias, tmp_mean, yerr=tmp_std, color=color, ls=':',capsize=2, alpha=.9, elinewidth=1.)
        ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color, alpha=0.3)
        if ii_cosmo_param == 0 and kk == 0:
            custom_lines1.append(mpl.lines.Line2D([0], [0], color='grey', ls=':', lw=1., marker=None, markersize=9))
            custom_labels1.append(mode_name)
        
legend = axs[0].legend(custom_lines, custom_labels, loc='upper right', fancybox=True, shadow=True, ncol=1,fontsize=12.)
axs[0].add_artist(legend)
        
legend1 = axs[1].legend(custom_lines1, custom_labels1, loc='upper right', fancybox=True, shadow=True, ncol=1,fontsize=12.)
axs[1].add_artist(legend1)
    
fig.set_tight_layout(True)
mpl.pyplot.show()

In [ ]:
selected_plot_keys = ['eagle-bahamas', 'v1-v2']
# selected_plot_keys = fraction_biased[mode_name].keys()

fontsize=26
fontsize1=18
fig, axs = plt.subplots(1, len(custom_titles), figsize=(4.2*len(custom_titles), 5.2))
axs[0].set_ylabel(r'Frac. bias no CL -' "\n" "Frac. bias CL", size=fontsize)

custom_lines = []
custom_labels = []
for ii_cosmo_param in range(len(custom_titles)):
    ax = axs[ii_cosmo_param]
    ax.set_xlim([1.5, 5.])
    ax.set_ylim([-1, 1])
    ax.set_title(custom_titles[ii_cosmo_param], size=fontsize+8, pad=16)
    ax.set_xlabel(r'$\sigma_\mathrm{thr.}$', size=fontsize)
    ax.axhline(0, c='k', ls=':', lw=0.9)
    if ii_cosmo_param != 0:
        ax.set_yticks([])
    
    mode_name = "no CL"
    mode_name1 = "CL Wein"
    for kk, name_load in enumerate(selected_plot_keys):
        color = colors[mode_name][name_load]
        try:
            tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
            tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
        
            tmp_mean1 = np.nanmean(fraction_biased[mode_name1][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
            tmp_std1 = np.nanstd(fraction_biased[mode_name1][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
            
            ax.scatter(thresholds_bias, tmp_mean1-tmp_mean, color=color, s=20)
            ax.errorbar(thresholds_bias, tmp_mean1-tmp_mean, yerr=((tmp_std1+tmp_std), (tmp_std1+tmp_std)),
                color=color, ls='', capsize=2, alpha=1., elinewidth=1.
            )
            ax.fill_between(
                thresholds_bias, (tmp_mean1-tmp_mean)-(tmp_std1+tmp_std), (tmp_mean1-tmp_mean)+(tmp_std1+tmp_std),
                color=color, alpha=0.3
            )
            if ii_cosmo_param == 0:
                custom_lines.append(mpl.lines.Line2D([0], [0], color=color, ls='-', lw=9, marker=None, markersize=9))
                custom_labels.append(name_load)
        except:
            print(custom_titles[ii_cosmo_param], ' - ERROR for ', name_load)
        

legend = axs[0].legend(custom_lines, custom_labels, loc='upper left', fancybox=True, shadow=True, ncol=3,fontsize=9.)
axs[0].add_artist(legend)

fig.set_tight_layout(True)
mpl.pyplot.show()

In [ ]:
ii_batch_sample = 0

fontsize=26
fontsize1=18
fig, axs = plt.subplots(1, len(custom_titles), figsize=(5.2*len(custom_titles), 5.2))
axs[0].set_ylabel(r'Normalized Counts ', size=fontsize)
for ii_cosmo_param in range(len(custom_titles)):
    
    tmp_prior_range = (list_range_priors[ii_cosmo_param][1]-list_range_priors[ii_cosmo_param][0])
    
    ax = axs[ii_cosmo_param]
    ax.set_title(custom_titles[ii_cosmo_param], size=fontsize+8, pad=16)
    ax.set_xlabel(r'$\mathrm{median}\left(\frac{2\sigma}{\Delta \mathrm{Prior}}\right)$', size=fontsize)
#     ax.set_xlim([0.0, max_err_hist[ii_cosmo_param]/tmp_prior_range])
    
    mode_name = 'no CL'
    name_load = 'eagle-bahamas'
    linestyle = '--'
    color0 = 'grey'
    color1 = '#D62728'
    color2 = '#8C564B'
    
#     ax.plot(
#         bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param]/tmp_prior_range,
#         y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param],
#         color=color0, lw=0.2, alpha=0.9, ls=linestyle
#     )
    tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
    tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.axvline(tmp_mean, color=color0, ls=linestyle, lw=2)
    ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color0)
    
    ii_stored_model_train = 1
#     ax.plot(
#         bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range,
#         y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param],
#         color=color1, lw=0.2, alpha=0.9, ls=linestyle
#     )
    tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.axvline(tmp_mean, color=color1, ls=linestyle, lw=2)
    ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color1)
    
    ii_stored_model_train = 2
#     ax.plot(
#         bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range,
#         y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param],
#         color=color2, lw=0.2, alpha=0.9, ls=linestyle
#     )
    tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.axvline(tmp_mean, color=color2, ls=linestyle, lw=2)
    ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color2)
        
        
    mode_name = "CL Wein"
    name_load = 'eagle-bahamas'
    linestyle = '-'
    
#     ax.plot(
#         bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param]/tmp_prior_range,
#         y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model, ii_cosmo_param],
#         color=color0, lw=0.2, alpha=0.9, ls=linestyle
#     )
    tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
    tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.axvline(tmp_mean, color=color0, ls=linestyle, lw=2)
    ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color0)
        
    ii_stored_model_train = 1
#     ax.plot(
#         bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range,
#         y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param],
#         color=color1, lw=0.2, alpha=0.9, ls=linestyle
#     )
    tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.axvline(tmp_mean, color=color1, ls=linestyle, lw=2)
    ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color1)
    
    ii_stored_model_train = 2
#     ax.plot(
#         bin_centers_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range,
#         y_hists_err[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param]/NN_points[mode_name][name_load][ii_batch_sample, ii_stored_model_train, ii_cosmo_param],
#         color=color2, lw=0.2, alpha=0.9, ls=linestyle
#     )
    tmp_mean = np.nanmean(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    tmp_std = np.nanstd(median_err[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param]/tmp_prior_range, axis=0)
    ax.axvline(tmp_mean, color=color2, ls=linestyle, lw=2)
    ax.axvspan(tmp_mean-tmp_std, tmp_mean+tmp_std, alpha=0.3, color=color2)

    
custom_lines = [
    mpl.lines.Line2D([0],[0],color=color0,ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color=color1,ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color=color2, ls='-',lw=8,marker=None,markersize=8)
]
custom_labels = [
    'Dset.-all',
    'Dset.-illsutris',
    'Dset.-eagle'
]
legend = axs[0].legend(custom_lines, custom_labels, loc='upper right', fancybox=True, shadow=True, ncol=1,fontsize=12)
axs[0].add_artist(legend)

custom_lines = [
    mpl.lines.Line2D([0],[0],color='grey',ls='--',lw=2,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='grey',ls='-',lw=2,marker=None,markersize=8)
]
custom_labels = [
    'No CL',
    'CL ' + name_load
]
legend = axs[-1].legend(custom_lines, custom_labels, loc='upper right', fancybox=True, shadow=True, ncol=1,fontsize=12)
axs[-1].add_artist(legend)


fig.set_tight_layout(True)
mpl.pyplot.show()

# Check results with box=5000

In [ ]:
models_path = '/cosmos_storage/home/dlopez/Projects/CL_inference/models_box_5000'

In [ ]:
fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors = cl_inference.evaluation_tools.load_numpy_stored_summaries(
    models_path, "no CL", {"illustris-eagle" : "only_inference_models_illustris_eagle_kmax_0.6"}, fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors, default_colors
)

fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors = cl_inference.evaluation_tools.load_numpy_stored_summaries(
    models_path, "CL Wein", {"illustris-eagle" : "only_inference_CL_Wein_models_illustris_eagle_kmax_0.6"}, fraction_biased, NN_points, bin_centers, y_hists, bin_centers_err, y_hists_err, median_err, std_err, colors, default_colors
)

In [ ]:
fontsize=26
fontsize1=18
fig, axs = plt.subplots(1, len(custom_titles), figsize=(4.2*len(custom_titles), 5.2))
axs[0].set_ylabel(r'Fraction biased points ', size=fontsize)
for ii_cosmo_param in range(len(custom_titles)):
    ax = axs[ii_cosmo_param]
    ax.set_xlim([1.5, 5.])
    ax.set_ylim([-0.1, 1])
    ax.set_title(custom_titles[ii_cosmo_param], size=fontsize+8, pad=16)
    ax.set_xlabel(r'$\sigma_\mathrm{thr.}$', size=fontsize)
    ax.axhline(0, c='k', ls=':', lw=0.9)
    if ii_cosmo_param != 0:
        ax.set_yticks([])
    
    mode_name = "no CL"
    name_load = "illustris-eagle"
    linestyle = '--'
    color0 = 'grey'
    color1 = '#9467BD'
    color2 = '#D62728'
    
    tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
    tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
    ax.plot(thresholds_bias, tmp_mean, color=color0, ls=linestyle, lw=2)
    ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color0, alpha=0.3)
    
    ii_stored_model_train = 1
    tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    ax.plot(thresholds_bias, tmp_mean, color=color1, ls='-', lw=2)
    ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color1, alpha=0.3)
    
    ii_stored_model_train = 2
    tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    ax.plot(thresholds_bias, tmp_mean, color=color2, ls='-', lw=2)
    ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color2, alpha=0.3)
    
        
    mode_name = "CL Wein"
    name_load = "illustris-eagle"
    linestyle = '-'
    
    tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
    tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
    ax.plot(thresholds_bias, tmp_mean, color=color0, ls='-', lw=2)
    ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color0, alpha=0.3)

    ii_stored_model_train = 1
    tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    ax.plot(thresholds_bias, tmp_mean, color=color1, ls='-', lw=2)
    ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color1, alpha=0.3)
    
    ii_stored_model_train = 2
    tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    ax.plot(thresholds_bias, tmp_mean, color=color2, ls='-', lw=2)
    ax.fill_between(thresholds_bias, tmp_mean-tmp_std, tmp_mean+tmp_std, color=color2, alpha=0.3)
        
custom_lines = [
    mpl.lines.Line2D([0],[0],color=color0,ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color=color1,ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color=color2, ls='-',lw=8,marker=None,markersize=8)
]
custom_labels = [
    'Dset.-all',
    'Dset.-illsutris',
    'Dset.-eagle'
]
legend = axs[0].legend(custom_lines, custom_labels, loc='upper right', fancybox=True, shadow=True, ncol=1,fontsize=12)
axs[0].add_artist(legend)

custom_lines = [
    mpl.lines.Line2D([0],[0],color='grey',ls='--',lw=2,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color='grey',ls='-',lw=2,marker=None,markersize=8)
]
custom_labels = [
    'No CL',
    'CL ' + name_load
]
legend = axs[-1].legend(custom_lines, custom_labels, loc='upper right', fancybox=True, shadow=True, ncol=1,fontsize=12)
axs[-1].add_artist(legend)
        
fig.set_tight_layout(True)
mpl.pyplot.show()

In [ ]:
fontsize=26
fontsize1=18
fig, axs = plt.subplots(1, len(custom_titles), figsize=(4.2*len(custom_titles), 5.2))
axs[0].set_ylabel(r'Frac. bias no CL -' "\n" "Frac. bias CL", size=fontsize)
for ii_cosmo_param in range(len(custom_titles)):
    ax = axs[ii_cosmo_param]
    ax.set_xlim([1.5, 5.])
    ax.set_ylim([-1, 1])
    ax.set_title(custom_titles[ii_cosmo_param], size=fontsize+8, pad=16)
    ax.set_xlabel(r'$\sigma_\mathrm{thr.}$', size=fontsize)
    ax.axhline(0, c='k', ls=':', lw=0.9)
    if ii_cosmo_param != 0:
        ax.set_yticks([])
    
    mode_name = "no CL"
    mode_name1 = "CL Wein"
    name_load = "illustris-eagle"
    color0 = 'grey'
    color1 = '#9467BD'
    color2 = '#D62728'
    
    tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
    tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
    tmp_mean1 = np.nanmean(fraction_biased[mode_name1][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
    tmp_std1 = np.nanstd(fraction_biased[mode_name1][name_load][:, ii_stored_model, ii_cosmo_param], axis=0)
    ax.scatter(thresholds_bias, tmp_mean1-tmp_mean, color=color0, s=20)
    ax.errorbar(thresholds_bias, tmp_mean1-tmp_mean, yerr=((tmp_std1+tmp_std), (tmp_std1+tmp_std)),
        color=color0, ls='', capsize=2, alpha=1., elinewidth=1.
    )
    ax.fill_between(
        thresholds_bias, (tmp_mean1-tmp_mean)-(tmp_std1+tmp_std), (tmp_mean1-tmp_mean)+(tmp_std1+tmp_std),
        color=color0, alpha=0.3
    )

    ii_stored_model_train = 1
    tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    tmp_mean1 = np.nanmean(fraction_biased[mode_name1][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    tmp_std1 = np.nanstd(fraction_biased[mode_name1][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    ax.scatter(thresholds_bias, tmp_mean1-tmp_mean, color=color1, s=20)
    ax.errorbar(thresholds_bias, tmp_mean1-tmp_mean, yerr=((tmp_std1+tmp_std), (tmp_std1+tmp_std)),
        color=color1, ls='', capsize=2, alpha=1., elinewidth=1.
    )
    ax.fill_between(
        thresholds_bias, (tmp_mean1-tmp_mean)-(tmp_std1+tmp_std), (tmp_mean1-tmp_mean)+(tmp_std1+tmp_std),
        color=color1, alpha=0.3
    )

    ii_stored_model_train = 2
    tmp_mean = np.nanmean(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    tmp_std = np.nanstd(fraction_biased[mode_name][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    tmp_mean1 = np.nanmean(fraction_biased[mode_name1][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    tmp_std1 = np.nanstd(fraction_biased[mode_name1][name_load][:, ii_stored_model_train, ii_cosmo_param], axis=0)
    ax.scatter(thresholds_bias, tmp_mean1-tmp_mean, color=color2, s=20)
    ax.errorbar(thresholds_bias, tmp_mean1-tmp_mean, yerr=((tmp_std1+tmp_std), (tmp_std1+tmp_std)),
        color=color2, ls='', capsize=2, alpha=1., elinewidth=1.
    )
    ax.fill_between(
        thresholds_bias, (tmp_mean1-tmp_mean)-(tmp_std1+tmp_std), (tmp_mean1-tmp_mean)+(tmp_std1+tmp_std),
        color=color2, alpha=0.3
    )
    
custom_lines = [
    mpl.lines.Line2D([0],[0],color=color0,ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color=color1,ls='-',lw=8,marker=None,markersize=8),
    mpl.lines.Line2D([0],[0],color=color2, ls='-',lw=8,marker=None,markersize=8)
]
custom_labels = [
    'Dset.-all',
    'Dset.-illsutris',
    'Dset.-eagle'
]
legend = axs[0].legend(custom_lines, custom_labels, loc='upper right', fancybox=True, shadow=True, ncol=1,fontsize=12)
axs[0].add_artist(legend)

fig.set_tight_layout(True)
mpl.pyplot.show()